In [1]:
import sys
sys.path.append("/home/alok/Programming/py-manifolds")

In [2]:
import math

import jax
import numpy as np

from manifolds.sphere import Sphere, StereographicChart, SpherePoint

In [3]:
sphere = Sphere(5)

In [4]:
northpole = SpherePoint(np.array([0., 0., 5.]))
southpole = SpherePoint(np.array([0., 0., -5.]))

In [5]:
south_projection = sphere.preferred_chart(northpole)
north_projection = sphere.preferred_chart(southpole)

In [28]:
northern_hemi_point = SpherePoint(np.array([0., 3., 4.]))
print(south_projection.point_to_coords(northern_hemi_point))
print(north_projection.point_to_coords(northern_hemi_point))
print(
    south_projection.coords_to_point(
        south_projection.point_to_coords(northern_hemi_point)
    ).coords
)

[0.         1.66666667]
[ 0. 15.]
[0. 3. 4.]


In [17]:
def south_to_north(c):
    return north_projection.point_to_coords(south_projection.coords_to_point(c))

In [18]:
jacobian = jax.jacfwd(south_to_north)

In [19]:
jacobian(south_projection.point_to_coords(northern_hemi_point)) @ np.array([1, 1])

DeviceArray([ 8.999998, -8.999992], dtype=float32)

In [9]:
15 / 1.6667

8.999820003599927

In [24]:
jax.jvp?

In [25]:
pt, tangent = jax.jvp(south_to_north, [south_projection.point_to_coords(northern_hemi_point)], [np.array([1., 1.])])

In [26]:
tangent

DeviceArray([ 8.999998, -8.999992], dtype=float32)